In [1]:
!pip install openai
!pip install googlesearch-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00


In [3]:
from openai import OpenAI, OpenAIError
from googlesearch import search
import getpass
# enter your own openai api key
api_key = ""
client = OpenAI(api_key = api_key)

Test Code

Role :

- 1.system : Role of chatgpt
- 2.user : you
- 3.assistant : background information

In [4]:
## Test
#reply = client.chat.completions.create(
  #model = "gpt-3.5-turbo",
  #messages = [{"role":"system","content": "Stock Consultant?"},{"role":"user","content": "TSMC's closing prices"}]
#)
#print(reply)
#print(reply.choices[0].message.content)
#print(reply.usage)
# for item in search("2024 NBS champion", advanced =True,num_results =3):
#   print(item.title)
#   print(item.description)
#   print(item.url)
#   print("\n").

ChatCode Start

In [5]:
from logging import error
hist = [] # conversation record for Assistant Use
search_hist=[] # search conversation record for Assistant Use
backtrace = 2 # conversation record size


def Reply_Message(messages):
  try:
    response = client.chat.completions.create(
        model = "gpt-3.5-turbo",

        messages = messages
    )
    reply = response.choices[0].message.content
  except OpenAIError as err:
    reply = f"{err.args[0]} 錯誤\n"
  return reply

def chat(sys_msg, user_msg):
  hist.append({"role":"user","content":user_msg})
  reply = Reply_Message(hist+[{"role":"system","content":sys_msg}])
  while len(hist) >= 2 * backtrace:
    hist.pop(0)
  hist.append({"role":"assistant","content":reply})
  return reply

  # history structure
  # - user : content
  # - assistant : reply


def search_chat(sys_msg, user_msg, search_g = True):
  web_res = []
  if search_g == True:
    content = "following are searching data:\n"
    for res in search(user_msg, advanced =True,num_results =3):
      content += f"title:{res.title}\n"\
             f"abstract:{res.description}\n\n"
    web_res=[{"role":"user","content" : content}]
  web_res.append({"role":"user","content":user_msg})
  while len(search_hist) >= 2 * backtrace:
    search_hist.pop(0) # remove oldest
  reply_full = ""
  for reply in Reply_Message(search_hist + web_res + [{"role":"system","content":sys_msg}]):
    reply_full += reply
    yield reply
  search_hist.append({"role":"user","content":user_msg})
  while len(search_hist) >= 2 * backtrace:
    search_hist.pop(0)
  search_hist.append({"role" : "assistant","content" : reply_full})


In [6]:
sys_msg = "user"
while True:
  msg = input("you say:")
  if not msg.strip():
    break
  for reply in search_chat(sys_msg, msg, search_g = True):
    print(reply, end = "")
  print('\n')

you say:do you know about Taiwan
Yes, I have information about Taiwan. Taiwan is an island nation located in East Asia, off the southeastern coast of China. It is officially known as the Republic of China. Taiwan has a rich history, vibrant culture, and is known for its technological advancements. Would you like to know more specific information about Taiwan?

you say:Yes, how big it is 
The size of a twin mattress is 38 inches by 75 inches. Twin size mattresses are the smallest standard mattress size and are typically intended for a single person.



KeyboardInterrupt: Interrupted by user